In [2]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings

In [3]:
warnings.filterwarnings("ignore")

## 2. 키워드 추출

#### [2-1) 텍스트 데이터 전처리]

In [4]:
# csv파일 불러오기
df = pd.read_csv('tweet_temp4.csv')
df.head()

,생성일,본문
0,2020-09-22 07:05:25,RT @littletwicesis: อัยต้าววววต้าวหมีทั้งสองน่...
1,2020-09-22 07:05:25,RT @iheregap: ก็มาดิวะ !!! https://t.co/b27IfK...
2,2020-09-22 07:05:25,"RT @cuzimafighter7: 👶hi, I’m *name*\n🐰hi *name..."
3,2020-09-22 07:05:25,고양이는 다른 포유류와 달리 단맛을 느끼도록 하는 유전자 일부가 손실돼 있기 때문에...
4,2020-09-22 07:05:25,RT @hanitweet: “남자가 어떻게 주사를 맞아. 나는 자궁이 없어.”\n“...


In [34]:
import re

# 텍스트 정제 함수 : 할글 이외의 문자는 전부 제거
def text_cleaning(text):
    hangul = re.compile('[^ㄱ-ㅣ가-힣]+') #한글의 정규표현식을 나타냅니다.
    result = hangul.sub('', text)
    return result

In [35]:
# 'tweet_text' 피처에 이를 적용합니다.
df['ko_text'] = df['본문'].apply(lambda x: text_cleaning(x))
df.head()

,본문,ko_text
0,신종 코로나바이러스 감염증(코로나19) 유행 이후 제약·바이오 업체의 '코로나19 ...,신종코로나바이러스감염증코로나유행이후제약바이오업체의코로나상술경쟁이끊이지않자국민에게헛된...
1,1일 제약바이오업계에 따르면 업체들은 코로나19 신약 개발이나 약물 재창출 연구와 ...,일제약바이오업계에따르면업체들은코로나신약개발이나약물재창출연구와관련된자료를쏟아내며포털의...
2,포털에 이름이 올라가면 주가가 급등하는 현상을 노리는 것이다.,포털에이름이올라가면주가가급등하는현상을노리는것이다
3,이에 제약사들은 전임상 단계인 동물실험이나 시험관 내 세포 실험(인 비트로) 중간결...,이에제약사들은전임상단계인동물실험이나시험관내세포실험인비트로중간결과까지앞다퉈발표한다
4,"급기야 의약품 사용 승인이 미뤄졌지만, 자체 성능 평가는 훌륭했다는 자기변명 식의 ...",급기야의약품사용승인이미뤄졌지만자체성능평가는훌륭했다는자기변명식의자료까지나온다


#### [2-2) KoNLPy를 이용한 키워드 추출 (형태소 분석기)]
#### 한국어 약식 불용어 사전 사용 (http://ranks.nl/stopwords/korean)

In [6]:
# csv파일 불러오기
df = pd.read_csv('news_data_ori_0701_0705.csv')
df.head()

print(df)


from konlpy.tag import Okt
from collections import Counter
from IPython.display import clear_output


korean_stopwords_path = './korean_stopwords.txt'
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

nouns_cnt = 1
count_data = []

def get_nouns(x):
    global nouns_cnt, count_data
    
    nouns_tagger = Okt()
    nouns = nouns_tagger.nouns(x)
    count_data.extend([z for y in nouns for z in y])

    #한글자 키워드를 제거합니다.
    nouns = [noun for noun in nouns if len(noun) > 1]

    #불용어를 제거합니다.
    nouns = [noun for noun in nouns if noun not in stopwords]

    clear_output(wait=True)
    print(f'형태소,불용어 제거중 .. {round((nouns_cnt / df.shape[0])*100,3)} %')
    nouns_cnt+=1
    
    return nouns


def get_nouns2(x):
    global nouns_cnt, count_data
    
    count_data.extend([y for x in nouns for y in x])
    
    
    clear_output(wait=True)
    print(f'형태소,불용어 제거중 .. {round((nouns_cnt / df.shape[0])*100,3)} %')
    nouns_cnt+=1
    
    return

                                                      본문
0       인천해양경찰서(서장 신동삼)는 도서 지역 치안 강화를 위해 인천 중부경찰서와 지난...
1       작년말 대비 두 배 가까이 늘어 지난달 26일 하룻새 4조 급증 SK바이오팜 청약...
2       ［중부매일 서인석 기자］ 이차영 괴산군수가 1일 열린 직원조회에서 민선 7기 2년...
3       라임자산운용(라임)의 1조6000억원 펀드 환매 중단 사건을 수사 중인 검찰이 1...
4       시모네타 소마루가 스위스 대통령 [EPA=연합뉴스 자료사진] . (제네바=연합뉴스...
...                                                  ...
21189   월요일인 6일은 전국이 가끔 구름이 많고 더운 가운데 소나기가 오는 곳이 있겠다....
21190   한국여자프로골프(KLPGA) 투어 맥콜-용평리조트 오픈 with SBS골프에서 김...
21191   [앵커] 추미애 법무부 장관의 '검·언 유착 의혹' 사건 지휘권 행사와 관련해, ...
21192   세계보건기구(WHO)가 말라리아 치료제 클로로퀸과 HIV(인간면역결핍바이러스) 치...
21193   '검언유착' 의혹 수사와 관련한 추미애 법무부 장관의 수사지휘에 윤석열 검찰총장이...

[21194 rows x 1 columns]


In [7]:
# df['nouns'] = df['ko_text'].apply(lambda x: get_nouns(x))
df['nouns'] = df['본문'].apply(lambda x: get_nouns(x))
print(df.shape)
df.head()

형태소,불용어 제거중 .. 100.0 %
(21194, 2)


,본문,nouns
0,인천해양경찰서(서장 신동삼)는 도서 지역 치안 강화를 위해 인천 중부경찰서와 지난...,"[인천, 해양경찰, 서장, 신동, 도서, 지역, 치안, 강화, 위해, 인천, 중부,..."
1,작년말 대비 두 배 가까이 늘어 지난달 26일 하룻새 4조 급증 SK바이오팜 청약...,"[작년, 말, 대비, 두, 배, 가까이, 지난달, 룻새, 급증, 바이오, 청약, 뭉..."
2,［중부매일 서인석 기자］ 이차영 괴산군수가 1일 열린 직원조회에서 민선 7기 2년...,"[중부, 매일, 서인석, 기자, 차영, 괴산군, 수가, 직원, 조회, 민선, 기, ..."
3,라임자산운용(라임)의 1조6000억원 펀드 환매 중단 사건을 수사 중인 검찰이 1...,"[라임, 자산운용, 라임, 펀드, 환매, 중단, 사건, 수사, 중인, 검찰, 신한은..."
4,시모네타 소마루가 스위스 대통령 [EPA=연합뉴스 자료사진] . (제네바=연합뉴스...,"[시모네타, 마루, 스위스, 대통령, 연합뉴스, 자료, 사진, 제네바, 연합뉴스, ..."


In [8]:
#빈도 수 구하기.

# count_data = [y for x in df['nouns'].tolist() for y in x]

from collections import Counter
count = Counter(count_data)

node_df = pd.DataFrame(count.items(), columns=['node', 'nodesize'])
# node_df = node_df[node_df['nodesize'] >= 50]
node_df.head()

,node,nodesize
0,인,84427
1,천,12763
2,해,79288
3,양,15673
4,경,61270


## 3. 연관 키워드 추출하기

### [트위터 연관 키워드 분석]

In [9]:
transactions = df['nouns'].tolist()
transactions = [transaction for transaction in transactions if transaction] # 공백 문자열을 방지합니다.and
# print(transactions)

In [10]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori as apriori2
from mlxtend.frequent_patterns import association_rules

te = TransactionEncoder()
te_result = te.fit(transactions).transform(transactions)

te_result

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [28]:
# keyword = '손흥민'

df = pd.DataFrame(te_result, columns=te.columns_)

itemset = apriori2(df, min_support=0.05, use_colnames=True)
itemset


# itemset = association_rules(itemset, metric="confidence", min_threshold=0.1)
# itemset

# itemset.shape

# itemset3 = association_rules(itemset2, metric="lift", min_threshold=0.1)


# 조건이 손흥민
# zz = itemset[itemset['antecedents'].apply(lambda x : (keyword in x) & (len(x) == 1))]
# zz = zz[zz['consequents'].apply(lambda x : (len(x) == 1))]


# 타겟이 손흥민
# zz = itemset[itemset['antecedents'].apply(lambda x : (len(x) == 1))]
# zz = zz[zz['consequents'].apply(lambda x : (keyword in x) & (len(x) == 1))]


# 전체 데이터
# zz = itemset[itemset['antecedents'].apply(lambda x :(len(x) == 1))]
# zz = zz[zz['consequents'].apply(lambda x :(len(x) == 1))]


# zz = itemset

# pd.set_option('display.max_rows', 1000)
# zz.sort_values(by=['support'], axis='index', ascending=False)

# zz



,support,itemsets
0,0.054119,(가격)
1,0.104464,(가능성)
2,0.179438,(가운데)
3,0.135982,(가장)
4,0.078985,(가족)
...,...,...
6118,0.060017,"(진자, 판정, 확진, 확, 코로나)"
6119,0.050344,"(진자, 발생, 확, 명, 감염, 코로나)"
6120,0.051005,"(감염증, 신종, 진자, 코로나바이러스, 확, 코로나)"
6121,0.052798,"(대한, 며, 수, 고, 말, 대해)"


In [14]:
itemset2 = association_rules(itemset, metric="confidence", min_threshold=0.0001)
itemset2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(가운데),(고),0.179438,0.582240,0.111305,0.620300,1.065367,0.006829,1.100236
1,(고),(가운데),0.582240,0.179438,0.111305,0.191167,1.065367,0.006829,1.014502
2,(수),(가운데),0.557328,0.179438,0.108521,0.194717,1.085153,0.008516,1.018974
3,(가운데),(수),0.179438,0.557328,0.108521,0.604786,1.085153,0.008516,1.120082
4,(감염),(코로나),0.108144,0.358309,0.101208,0.935864,2.611891,0.062459,10.005142
...,...,...,...,...,...,...,...,...,...
1683,"(수, 고)","(며, 지난)",0.373360,0.161602,0.100500,0.269177,1.665677,0.040164,1.147197
1684,(며),"(수, 지난, 고)",0.368878,0.162593,0.100500,0.272448,1.675643,0.040523,1.150993
1685,(지난),"(며, 수, 고)",0.391054,0.222469,0.100500,0.256998,1.155210,0.013503,1.046473
1686,(수),"(며, 지난, 고)",0.557328,0.144192,0.100500,0.180325,1.250592,0.020138,1.044083


In [1]:
# 조건이 키워드
keyword = '코로나'
zz = itemset2[itemset2['antecedents'].apply(lambda x : (keyword in x) & (len(x) == 1))]
zz = zz[zz['consequents'].apply(lambda x : (len(x) == 1))]

pd.set_option('display.max_rows', 1000)
zz.sort_values(by=['lift','support','confidence'], axis='index', ascending=False)


NameError: name 'itemset2' is not defined